# Annex A Execution

A notebook implementing the execution rules from KerML Annex A with PyMBE.

In [ ]:
import json
import pymbe.api as pm

import copy

from importlib import resources as lib_resources

from pathlib import Path

from typing import Any, Collection, Dict, List, Tuple, Union

from pymbe.model import Model, Element
from pymbe.model_modification import *

from pymbe.query.metamodel_navigator import is_type_undefined_mult, \
                                    is_multiplicity_one, \
                                    is_multiplicity_specific_finite, \
                                    get_finite_multiplicity_types, \
                                    identify_connectors_one_side, \
                                    get_lower_multiplicity, \
                                    get_upper_multiplicity, \
                                    does_behavior_have_write_features, \
                                    get_most_specific_feature_type, \
                                    has_type_named, \
                                    get_effective_lower_multiplicity, \
                                    get_feature_bound_values, \
                                    get_more_general_types

from pymbe.metamodel import derive_inherited_featurememberships

from pymbe.text_concrete_syntax import serialize_kerml_atom

from pymbe.interpretation.working_maps import FeatureTypeWorkingMap
from pymbe.interpretation.execute_kerml_atoms import KermlForwardExecutor

from uuid import uuid4

## Key Helpers for the Algorithm

These helpers are yet to be implemented in the core of the Python tool and thus need to be more spelled out.

### Check for Connectors to Features

In [ ]:
def is_feature_connected(feature):
    print(f"...Inspecting {feature.declaredName} for connector references.")
    if hasattr(feature, "reverseReferenceSubsetting"):
        print(f"...Found link to connector end {feature.reverseReferenceSubsetting[0]}")
        return True
    else:
        print(f"...Found no reverse edge outgoing to connector end.")
        return False

# Load up Kernel Libraries

Load up the model libraries into memory so that key features for subsetting can be found.

In [ ]:
library = "KernelLibrary"

library_model = None

with lib_resources.path("pymbe.static_data", "KernelLibrary.json") as lib_data:
    library_model = pm.Model.load_from_post_file(lib_data)

## Routines for Execution

The following sections are focused on solving the problem of mapping values to KerML types in the model. The approach taken here is to find one legal set of values for types in the model via an approach where the program will walk straight ahead in the model, deriving values as it goes. This approach is called "execution" here.

In [ ]:
def print_values_dictionary(model, values_dict):
    print_string = ""
    for k, v in values_dict.items():
        print_string = print_string + f">>>Key {model.get_element(k)} ({k}) has values {v}\n"
        
    print(print_string)

In [ ]:
def generate_values_for_type_annex_a(input_model,
                                     package_to_execute,
                                     package_to_populate,
                                     type_to_value,
                                     atom_index,
                                     bound_features_to_atom_values_dict):
    
    """
    Generate values for the given single type using the Annex A rules
    
    bound_features_to_atom_values_dict - pass as a mutable object, update as we go
    """
    
    print(f"Applying Annex A atom algorithm to {type_to_value.basic_name}")
    
    new_classifier = None
    
    # TODO: Create map from new_classifier to the features discovered under this type rather than having the dictionary key to features globally
    
    if type_to_value._metatype in classifier_metas() and not has_type_named(type_to_value, "FeatureWritePerformance"):
    
        # Step 1 - create new atom from the classifier
        base_name = type_to_value.basic_name
        new_classifier = build_from_classifier_pattern(
            owner=package_to_populate,
            name=f"My{base_name}",
            model=input_model,
            specific_fields={},
            metatype=type_to_value._metatype,
            superclasses=[type_to_value]
        )
        
        print(f"Executing step 1. Working from {base_name} to create {new_classifier}")
    
    # use derived property 'feature' to get all features including the inherited ones
    candidate_features = type_to_value.feature
    
    try:
        #print(f"Trying to find metatype for {type_to_value.throughFeatureTyping}")
        featured_meta = type_to_value.throughFeatureTyping[0]._metatype
        print(f"Type {type_to_value.basic_name} is has a type of metatype {featured_meta}.")
        if featured_meta in datatype_metas():
            print(f"Type {type_to_value.basic_name} is typed by a datatype. Bypassing further elaboration.")
            return bound_features_to_atom_values_dict
    except:
        pass
    
    print(f"...Found features {candidate_features} under the type to value {type_to_value.basic_name}.")
    
    # set up for multiple passes on the feature list, will test per pass to determine whether to handle it
    
    # TODO: Instead of this testing approach, gather all features and allocated into sub-collections that
    # handle these passes
    
    passes = ['Non-connector Features', 'FeatureWritePerformances', 'Connector Features']
    
    for pass_number, pass_kind in enumerate(passes):
        
        print(f"Currently on pass {pass_kind} under {type_to_value}.")
        print(f"Current values dict at this step is")
        print_values_dictionary(input_model, bound_features_to_atom_values_dict)
    
        for cf in candidate_features:
            
            if has_type_named(cf, "FeatureWritePerformance") and not pass_kind == 'FeatureWritePerformances':
                continue
            if cf._metatype not in connector_metas() and not pass_kind == 'Non-connector Features':
                if not has_type_named(cf, "FeatureWritePerformance"):
                    continue
            if cf._metatype in connector_metas() and not pass_kind == 'Connector Features':
                continue
            
            # common pre-processing steps
            
            cf_name = cf.basic_name
            
            lm = get_effective_lower_multiplicity(cf)
            
            if lm > -1:
                # need to test multiplicity
                print(f"...Found effective lower multiplicity of {cf_name} ({cf._id}) as {lm}.")
            elif cf._metatype not in connector_metas():
                print(f"...{cf_name} ({cf._id}) has unbounded multiplicity. Skipping.")
                continue
            
            redefining_features = set(cf.reverseRedefinition)
            cf_redefined = False
            
            for rf in redefining_features:
                print(f"...Discovered that {cf} ({cf._id}) is redefined by {rf} ({rf._id})! Skipping.")
                cf_redefined = True
                
            if cf_redefined:
                continue
            
            # Step 2 - find Features with lower multiplicity > 0 that are not connectors
            
            # Look for existing values for the feature or previous atom assignment
            
            values_set_in_model = find_model_existing_values_for_feature(cf, bound_features_to_atom_values_dict)
            
            # Pass-specific core steps
            
            if has_type_named(cf, "FeatureWritePerformance"):
                print(f"Feature {cf} has type of FeatureWritePerformance")
                if pass_kind == 'FeatureWritePerformances':
                    create_feature_write_performance_atoms(
                        input_model,
                        bound_features_to_atom_values_dict,
                        cf,
                        package_to_populate
                    )
            
            elif cf._metatype not in connector_metas() and pass_kind == 'Non-connector Features':
                
                handled_as_self_reference = handle_self_referencing_feature(cf, bound_features_to_atom_values_dict)
                
                if len(values_set_in_model) == 0 and handled_as_self_reference == False:
                    # Step 3 - create new Atoms to go with the Features
                    
                    used_typ = get_most_specific_feature_type(cf)
                    
                    if used_typ._metatype in datatype_metas():
                        print(f"Executing step 2. Identified {cf_name} ({cf._id}) as a non-connector Feature. No existing values found. " + \
                              f"This is a datatype. Skipping generation of values.")
                        continue
                    else:
                        print(f"Executing step 2. Identified {cf_name} ({cf._id}) as a non-connector Feature. No existing values found. " + \
                              f"Generating {lm} new values specializing type {used_typ}.")
                    
                    for i in range(0, lm):
                        inspect_type_for_atom_rev_2(input_model,
                          package_to_execute,
                          package_to_populate,
                          i,
                          cf,
                          used_typ,
                          bound_features_to_atom_values_dict
                         )
                                
            if cf._metatype in connector_metas() and pass_kind == 'Connector Features':
                print(f"Executing step 4. Identified {cf.basic_name} as a Connector")

                # check the feature ends of the connector
                
                used_typ = get_most_specific_feature_type(cf)
                
                connector_atoms = []

                connector_ends = cf.throughEndFeatureMembership

                lm_end1 = get_effective_lower_multiplicity(connector_ends[0])
                lm_end2 = get_effective_lower_multiplicity(connector_ends[1])

                if lm_end1 == 1 and lm_end2 == 1:
                    print(f"Executing step 5. Identified {cf} as a Connector with 1-to-1 ends")
                    
                    end_connected_mults = [get_effective_lower_multiplicity(connector_ends[i].throughReferenceSubsetting[0]) for i in range(0,2)]
                    
                    # get the multiplicity of the ends
                    #end1_connected_mult = get_effective_lower_multiplicity(connector_ends[0].throughReferenceSubsetting[0])
                    #end2_connected_mult = get_effective_lower_multiplicity(connector_ends[1].throughReferenceSubsetting[0])
                    
                    for i, end_connected_mult in enumerate(end_connected_mults):
                    
                        print(f"...Effective lower multiplicity of {connector_ends[i].throughReferenceSubsetting[0]}, bound to assoc feature {connector_ends[i]}, " + \
                              f"is {end_connected_mult}")

                    # check to see if the already built instances have a finite value
                    
                    print(f"...Looking for connected end feature values filled in during execution.")
                    
                    found_values = []
                    
                    ends_to_process = []

                    if hasattr(cf, "throughEndFeatureMembership"):
                        for cf_ele in cf.throughEndFeatureMembership:
                            ends_to_process.append(cf_ele)
                    if used_typ is not None and hasattr(used_typ, "throughEndFeatureMembership"):
                        for cf_ele in used_typ.throughEndFeatureMembership:
                            ends_to_process.append(cf_ele)
                    
                    for con_end in ends_to_process:
                        bound_feature = con_end.throughReferenceSubsetting[0]
                        if bound_feature._id in bound_features_to_atom_values_dict:
                            print(f"...Found connected end feature values filled in during execution {bound_features_to_atom_values_dict[bound_feature._id]}!")
                            found_values.append(len(bound_features_to_atom_values_dict[bound_feature._id]))
                                                          
                    number_to_make = max(end_connected_mults + found_values)
                    
                    print(f"...Found that number of atoms to make for {cf} is {number_to_make}")

                    feature_value_atoms = []

                    used_typ = None

                    for i in range(0, number_to_make):
                        print(f"Executing step 5b. Creating atom #{i + 1} to be value for {cf}")

                        typ = []
                        used_typ = None
                        used_name = cf.basic_name
                        used_metatype = "Association"
                        
                        elaboration_end = False

                        if hasattr(cf, "throughFeatureTyping"):
                            typ = cf.throughFeatureTyping

                        if len(typ) > 0:
                            used_typ = typ[0]
                            used_name = used_typ.basic_name
                            used_metatype = used_typ._metatype

                        #feature_value_atoms.append(new_ft_classifier)

                        # need to do this for Association types and also nested end features

                        #ends_to_process = []

                        #if hasattr(cf, "throughEndFeatureMembership"):
                        #    for cf_ele in cf.throughEndFeatureMembership:
                        #        ends_to_process.append(cf_ele)
                        #if used_typ is not None and hasattr(used_typ, "throughEndFeatureMembership"):
                        #    for cf_ele in used_typ.throughEndFeatureMembership:
                        #        ends_to_process.append(cf_ele)

                        for con_end in ends_to_process:
                            print(f"...Inspecting {con_end} for connected features.")
                            if len(con_end.throughReferenceSubsetting) > 0:
                                bound_feature = con_end.throughReferenceSubsetting[0]
                                bound_feature_type = bound_feature.throughFeatureTyping

                                print(f"...Found connected feature {bound_feature}.")

                                if len(bound_feature_type) > 0:
                                    feature_used_type = bound_feature_type[0]

                                    if bound_feature._id in bound_features_to_atom_values_dict:
                                        # if not enough atoms exist, make some more
                                        if (i + 1) > len(bound_features_to_atom_values_dict[bound_feature._id]):
                                            print(f"Executing step 5b (1-to-1 variant). Creating atom #{i + 1} to " + \
                                                f"be value for {con_end} and also {bound_feature} to fill in rest of values.")
                                            
                                            inspect_type_for_atom_rev_2(input_model,
                                                                      package_to_execute,
                                                                      package_to_populate,
                                                                      i,
                                                                      bound_feature,
                                                                      feature_used_type,
                                                                      bound_features_to_atom_values_dict
                                                                     )
                                            
                                    else:
                                        print(f"Executing step 5b (1-to-1 variant). Creating atom #{i + 1} to " + \
                                            f"be value for {con_end} and also {bound_feature}")
                                        
                                        if has_type_named(bound_feature, "FeatureWritePerformance"):
                                            create_feature_write_performance_atoms(
                                                input_model,
                                                bound_features_to_atom_values_dict,
                                                bound_feature,
                                                package_to_populate
                                            )
                                        
                                        else:
                                            inspect_type_for_atom_rev_2(input_model,
                                                                      package_to_execute,
                                                                      package_to_populate,
                                                                      i,
                                                                      bound_feature,
                                                                      feature_used_type,
                                                                      bound_features_to_atom_values_dict            
                                                                     )
                                        

                        if len(ends_to_process) == 2:
                            source_end = ends_to_process[0]
                            target_end = ends_to_process[1]

                            source_bound_feature = source_end.throughReferenceSubsetting[0]
                            target_bound_feature = target_end.throughReferenceSubsetting[0]
                            
                            try:
                                source_atom = bound_features_to_atom_values_dict[source_bound_feature._id][i]
                            except KeyError:
                                raise KeyError(f"...Failed to find atoms for {source_bound_feature} and connector {cf}." + \
                                               f"{bound_features_to_atom_values_dict}")
                            
                            try:
                                target_atom = bound_features_to_atom_values_dict[target_bound_feature._id][i]
                            except KeyError:
                                raise KeyError(f"...Failed to find atoms for {target_bound_feature} and connector {cf}." + \
                                               f"{bound_features_to_atom_values_dict}")

                            print(f"...Typing atom association ends from {source_atom} to {target_atom} under " + 
                                 f"{used_name}{i + 1} to specialize {[ft.basic_name for ft in cf.throughFeatureTyping]}")
                            
                            # check for typing
                            
                            cn_types = cf.throughFeatureTyping
                            
                            if len(cf.throughFeatureTyping) == 0:
                                pass

                            new_cn_classifier = build_from_binary_assoc_pattern(
                                name=f"{used_name}{i + 1}",
                                source_role_name=connector_ends[0].basic_name,
                                target_role_name=connector_ends[1].basic_name,
                                source_type=source_atom,
                                target_type=target_atom,
                                model=input_model,
                                metatype=used_metatype,
                                owner=package_to_populate,
                                superclasses=cf.throughFeatureTyping,
                                specific_fields={}
                            )
                            
                            if cf._id in bound_features_to_atom_values_dict:
                                bound_features_to_atom_values_dict[cf._id].append(new_cn_classifier)
                            else:
                                bound_features_to_atom_values_dict.update({cf._id: [new_cn_classifier]})
                            
                            connector_atoms.append(new_cn_classifier)
                        
            
    #if type_to_value._metatype in feature_metas():
        # Step 1 - create new atom from the classifier
        
    #    try:
            #print(f"Trying to find metatype for {type_to_value.throughFeatureTyping}")
    #        feature_type = get_most_specific_feature_type(type_to_value)
            #feature_type = type_to_value.throughFeatureTyping[0]
    #        print(f"Found type {feature_type} for {type_to_value}.")
            
    #        base_name = feature_type.basic_name
    #        new_classifier = build_from_classifier_pattern(
    #            owner=package_to_populate,
    #            name=f"My{base_name}",
    #            model=input_model,
    #            specific_fields={},
    #            metatype=type_to_value._metatype,
    #            superclasses=[type_to_value]
    #        )
            
    #        print(f"Executing step 1. Working from {base_name} to create {new_classifier.declaredName}") 
            
    #    except:
    #        print(f"Cannot find the feature type for {type_to_value.basic_name}")
        
    if type_to_value._metatype in datatype_metas():
        print(f"Classifier {type_to_value.basic_name} is a datatype. Bypassing further elaboration.")
        return bound_features_to_atom_values_dict
    
    elif get_most_specific_feature_type(type_to_value) is not None:
        
        try:
            if get_most_specific_feature_type(type_to_value)._metatype in datatype_metas():
                print(f"Classifier {type_to_value.basic_name} is a datatype or typed by one. Bypassing further elaboration.")
                return bound_features_to_atom_values_dict
        except:
            pass
            
    print(f"Current values dict is")
    print_values_dictionary(input_model, bound_features_to_atom_values_dict)
    
    if len(bound_features_to_atom_values_dict.keys()) == 0:
        return {}
    
    if new_classifier == None:
        print(f"...No classifiers created for {type_to_value}")
        try:
            new_classifier = bound_features_to_atom_values_dict[type_to_value._id][atom_index]
            print(f"...Classifier for {type_to_value} found in bound values dict as {new_classifier}")
        except KeyError:
            pass
    else:
        print(f"...Classifier {new_classifier} created for {type_to_value}")
    
    embed_values_in_model_atom_classifier_style(
        input_model,
        bound_features_to_atom_values_dict,
        type_to_value,
        new_classifier,
        package_to_populate
    )
    
    return bound_features_to_atom_values_dict
    
    #print(f"Bound features dict: \n{bound_features_to_atom_values_dict}")

## Atom Metadata Load

Bring up the Atom metadata.

filename = "A-2-Atoms"

if not filename.endswith(".json"):
    filename += ".json"

json_file = Path(Path.cwd()) / "annex_a_data" / filename

atoms_data = pm.Model.load_from_post_file(json_file)
atoms_data

## Annex A.3.2 Without Connectors Case

In [ ]:
filename = "A-3-2-WithoutConnectors"

if not filename.endswith(".json"):
    filename += ".json"

json_file = Path(Path.cwd()) / "annex_a_data" / filename

without_connectors_data = pm.Model.load_from_post_file(json_file)
without_connectors_data

without_connectors_data.reference_other_model(library_model)

In [ ]:
packages = [ele for ele in without_connectors_data.elements.values() if ele._metatype == 'Package']
packages

In [ ]:
packages = [ele for ele in one_2_one_connectors_data.elements.values() if ele._metatype == 'Package']
packages

In [ ]:
one_2_one_connectors_executor = KermlForwardExecutor(one_2_one_connectors_data, packages[3])

In [ ]:
without_connectors_to_execute_classifiers = \
    [ele for ele in packages[0].throughOwningMembership if ele._metatype == 'Classifier']

In [ ]:
without_connectors_executor = KermlForwardExecutor(without_connectors_data, packages[1])

In [ ]:
packages[0].throughOwningMembership

In [ ]:
packages[0].throughOwningMembership[1].throughSubclassification

In [ ]:
packages[0].throughOwningMembership[2].basic_name

In [ ]:
for item in packages[1].throughOwningMembership:
    print(serialize_kerml_atom(item))

## Annex A.3.3 One-To-One Connectors Case

In [ ]:
filename = "A-3-3-OneToOneConnectors"

if not filename.endswith(".json"):
    filename += ".json"

json_file = Path(Path.cwd()) / "annex_a_data" / filename

one_2_one_connectors_data = pm.Model.load_from_post_file(json_file)
one_2_one_connectors_data

one_2_one_connectors_data.reference_other_model(library_model)

In [ ]:
packages = [ele for ele in one_2_one_connectors_data.elements.values() if ele._metatype == 'Package']
packages

In [ ]:
packages[2].throughOwningMembership

In [ ]:
packages[2].throughOwningMembership[1].throughFeatureMembership[0].throughEndFeatureMembership

In [ ]:
one_2_one_connectors_executor = KermlForwardExecutor(one_2_one_connectors_data, packages[3])

In [ ]:
one_2_one_connectors_executor.execute_from_classifier(packages[2].throughOwningMembership[1])

In [ ]:
for item in packages[3].throughOwningMembership:
    print(serialize_kerml_atom(item))

In [ ]:
packages[3].throughOwningMembership

In [ ]:
packages[3].throughOwningMembership[4].throughSubclassification

## Annex A.3.6 Timing for Behaviors, Sequences

In [ ]:
filename = "A-3-6-Sequences"

if not filename.endswith(".json"):
    filename += ".json"

json_file = Path(Path.cwd()) / "annex_a_data" / filename

sequences_data = pm.Model.load_from_post_file(json_file)
sequences_data

sequences_data.reference_other_model(library_model)

In [ ]:
packages = [ele for ele in sequences_data.elements.values() if ele._metatype == 'Package']
packages

In [ ]:
packages[0].throughOwningMembership

In [ ]:
packages[0].throughOwningMembership[1].throughFeatureMembership

In [ ]:
packages[0].throughOwningMembership[1].throughFeatureMembership[0].throughSubsetting[0]._derived

In [ ]:
sequences_executor = KermlForwardExecutor(sequences_data, packages[1])

In [ ]:
sequences_executor.execute_from_classifier(packages[0].throughOwningMembership[1])

In [ ]:
for item in packages[1].throughOwningMembership:
    print(serialize_kerml_atom(item))

In [ ]:
packages[1].throughOwningMembership

## Annex A.3.8 Feature Value Changes

In [ ]:
filename = "A-3-8-ChangingFeatureValues"

if not filename.endswith(".json"):
    filename += ".json"

json_file = Path(Path.cwd()) / "annex_a_data" / filename

values_data = pm.Model.load_from_post_file(json_file)
values_data

values_data.reference_other_model(library_model)

In [ ]:
packages = [ele for ele in values_data.elements.values() if ele._metatype == 'Package']
packages

In [ ]:
packages[1]

In [ ]:
values_executor = KermlForwardExecutor(values_data, packages[1])

In [ ]:
packages[0]

In [ ]:
has_type_named(packages[0].throughOwningMembership[3].throughFeatureMembership[1], "FeatureWritePerformance")

In [ ]:
packages[0].throughOwningMembership[3].throughFeatureMembership[1].throughFeatureMembership

In [ ]:
packages[0].throughOwningMembership[3].throughFeatureMembership[1].throughFeatureMembership[0].throughFeatureMembership[0].throughFeatureMembership[0].throughSubsetting

In [ ]:
values_executor.execute_from_classifier(packages[0].throughOwningMembership[1])

In [ ]:
for item in packages[1].throughOwningMembership:
    print(serialize_kerml_atom(item))

In [ ]:
packages[1].throughOwningMembership[3]

In [ ]:
packages[1].throughOwningMembership[3].throughFeatureMembership[19]

In [ ]:
packages[1].throughOwningMembership[3].throughFeatureMembership[19]._derived

In [ ]:
packages[1].throughOwningMembership[3].throughFeatureMembership[19].throughFeatureValue[0]

In [ ]:
packages[1].throughOwningMembership[3].throughFeatureMembership[19].throughFeatureChaining[0].throughFeatureMembership

In [ ]:
packages[1].throughOwningMembership[3].throughFeatureMembership[19].throughFeatureChaining[0].throughParameterMembership[0].throughFeatureValue[0].throughMembership[0].basic_name

In [ ]:
expression = packages[1].throughOwningMembership[3].throughFeatureMembership[19].throughFeatureChaining[0]
expression

In [ ]:
# first item will be FRE to another feature
expression_label = (
    expression.throughParameterMembership[0]
    .throughFeatureValue[0]
    .throughMembership[0]
    .basic_name
)
# check if this is a two-item feature chain or n > 2
if "throughMembership" in expression._derived and len(expression.throughMembership) > 0:
    # if hasattr(expression, "throughMembership"):
    # this is the n = 2 case
    second_item = expression.throughMembership[0].basic_name
    expression_label += f".{second_item}"
else:
    # this is the n > 2 case
    chains = expression.throughOwningMembership[0].throughFeatureChaining
    other_items = ".".join([chain.basic_name for chain in chains])
    expression_label += f".{other_items}"

In [ ]:
expression_label

In [ ]:
packages[1].throughOwningMembership[3].throughFeatureMembership[19].throughFeatureChaining[0].throughOwningMembership[0].throughFeatureChaining

In [ ]:
packages[1].throughOwningMembership[3].throughFeatureMembership[19].throughFeatureChaining[0]

In [ ]:
packages[1].throughOwningMembership[3].throughFeatureMembership[19].throughFeatureChaining[0].throughMembership

In [ ]:
packages[1].throughOwningMembership[3].throughFeatureMembership[19].throughFeatureValue[0]._derived

In [ ]:
values_data._add_labels(packages[1].throughOwningMembership[3].throughFeatureMembership[19].throughFeatureValue[0])

In [ ]:
mypaint1 = get_more_general_types(packages[1].throughOwningMembership[2],0,100)
mypaint1

In [ ]:
mypaint1[1]._data

In [ ]:
derive_inherited_featurememberships(packages[0].throughOwningMembership[3].throughFeatureMembership[0].throughFeatureTyping[0])

In [ ]:
packages[0].throughOwningMembership[3].throughFeatureMembership[0].throughFeatureTyping[0]

In [ ]:
packages[0].throughOwningMembership[3].throughFeatureMembership[0]